In [20]:
import azure.storage.file.models, os, time
from azure.storage.file import FileService

In [21]:
account_name="ntfisheriesstoreeastdev"
account_key="M8KfGcKvH49vaSyn/6xhsTkvrvZ6zjFVM/3Er6s0z3+PH89fKNf3F20kgLCzgnU+5IouIvMTCP2BHjBRjvUkww=="
model_training_share ="modeltraining"
model_weights_share = 'modelweights'

In [22]:
file_service = FileService(account_name=account_name, account_key=account_key)

In [23]:
if not file_service.exists(model_training_share):
    file_service.create_share(model_training_share)

In [24]:
if file_service.exists(model_weights_share):
    file_service.create_share(model_weights_share)

In [25]:
def download_file(share_name, directory_name, file_name, file_path, overwrite=False):
    if(not os.path.exists(file_path) or overwrite):
        print("Copying to %s" %file_path)
        file_service.get_file_to_path(share_name, directory_name, file_name, file_path, open_mode='wb')

In [26]:
def download_files(share_name, directory_name, download_path, overwrite=False):
    if not os.path.exists(download_path):
        os.makedirs(download_path)
        
    generator = file_service.list_directories_and_files(share_name, directory_name)
    
    for file_or_dir in generator:
        file_path = os.path.join(download_path, file_or_dir.name)
        if(not os.path.exists(file_path) or overwrite):
            print("Copying to %s" %file_path)
            file_service.get_file_to_path(share_name, directory_name, file_name= file_or_dir.name, file_path=file_path, open_mode='wb')

In [27]:
def list_files(share_name, directory_name=None):
    generator = file_service.list_directories_and_files(share_name, directory_name)

    for file_or_dir in generator:
        print(file_or_dir.name)

In [28]:
def upload_files(path, share_folder, overwrite=False):
     for root, dirs, files in os.walk(path):
            for file in files:
                sections = (root.split('/'))
                parent = sections[-1]
                file_service.create_directory(share_folder, parent)
                if(overwrite or not file_service.exists(share_folder, directory_name=parent, file_name=file)):
                    print("Uploading %s parent:%s file:%s" %(root, parent, file))
                    file_service.create_file_from_path(share_name=share_folder,
                                                       directory_name=parent,
                                                       file_name=file,
                                                       local_file_path=os.path.join(root,file))